In [1]:
lambda_vals = [415, 445, 480, 515, 555, 590, 630, 680, 910]
f_vals      = [453, 588, 765, 931, 5334, 2852, 1057, 1111, 1282]

integral = 0.0

for i in range(len(lambda_vals) - 1):
    delta_lambda = lambda_vals[i + 1] - lambda_vals[i]
    avg_f = (f_vals[i] + f_vals[i + 1]) / 2
    area = delta_lambda * avg_f
    integral += area

print(integral)


745102.5


In [4]:
import numpy as np
numpy_val = np.trapezoid(f_vals, x=lambda_vals)

print(numpy_val)

745102.5


In [5]:
import pandas as pd

# Read the Excel file
file_path = 'data/raw/raw_light_sensor_values_compiled_microclimate_sensor_data_MK_20251118_irr_lux 1.xlsx'
df = pd.read_excel(file_path)

# Display basic information about the dataset
print("Dataset shape:", df.shape)
print("\nColumn names:")
print(df.columns.tolist())
print("\nFirst few rows:")
print(df.head())
print("\nData types:")
print(df.dtypes)


Dataset shape: (252153, 44)

Column names:
['datetime', 'date', 'time', 'year', 'month', 'day', 'hour', 'minute', 'second', 'area', 'eu', 'SHT0_temp', 'SHT0_humidity', 'SHT0_F1', 'SHT0_F2', 'SHT0_F3', 'SHT0_F4', 'SHT0_F5', 'SHT0_F6', 'SHT0_F7', 'SHT0_F8', 'SHT0_F9', 'SHT0_F10', 'SHT0_F11', 'SHT0_F12', 'SHT1_temp', 'SHT1_humidity', 'SHT1_F1', 'SHT1_F2', 'SHT1_F3', 'SHT1_F4', 'SHT1_F5', 'SHT1_F6', 'SHT1_F7', 'SHT1_F8', 'SHT1_F9', 'SHT1_F10', 'SHT1_F11', 'SHT1_F12', 'file', 'sht0_irradiance(μW/cm²)', 'sht0_lux', 'sht1_irradiance(μW/cm²)', 'sht1_lux']

First few rows:
             datetime       date      time  year  month  day  hour  minute  \
0 2025-08-12 17:07:56 2025-08-12  17:07:56  2025      8   12    17       7   
1 2025-08-12 17:17:57 2025-08-12  17:17:57  2025      8   12    17      17   
2 2025-08-12 17:27:58 2025-08-12  17:27:58  2025      8   12    17      27   
3 2025-08-12 17:37:59 2025-08-12  17:37:59  2025      8   12    17      37   
4 2025-08-12 17:48:00 2025-08-12  17:48

In [6]:
# Search for the row with date 8/12/2025 and eu_5
# Try different date formats
target_date = '8/12/2025'
target_date_alt = '2025-08-12'
target_date_alt2 = '08/12/2025'

# Find rows matching the date
matching_rows = None

# Check each date column
for col in df.columns:
    if 'date' in col.lower() or 'time' in col.lower():
        # Try to convert to datetime
        try:
            df[col] = pd.to_datetime(df[col], errors='coerce')
            # Check for the target date
            mask = df[col].dt.strftime('%m/%d/%Y') == target_date
            if mask.any():
                matching_rows = df[mask]
                print(f"Found {len(matching_rows)} row(s) with date {target_date} in column '{col}'")
                break
        except:
            # Try string matching
            mask = df[col].astype(str).str.contains(target_date, na=False) | \
                   df[col].astype(str).str.contains(target_date_alt, na=False) | \
                   df[col].astype(str).str.contains(target_date_alt2, na=False)
            if mask.any():
                matching_rows = df[mask]
                print(f"Found {len(matching_rows)} row(s) with date {target_date} in column '{col}'")
                break

if matching_rows is None or len(matching_rows) == 0:
    print(f"No rows found with date {target_date}")
    print("\nTrying to find any row with 'eu_5'...")
    # Try to find eu_5 in any column
    for col in df.columns:
        mask = df[col].astype(str).str.contains('eu_5', case=False, na=False)
        if mask.any():
            print(f"Found 'eu_5' in column '{col}':")
            print(df[mask].head())
            break
else:
    # Now filter for eu_5
    print(f"\nFiltering for 'eu_5'...")
    eu_5_rows = None
    for col in matching_rows.columns:
        mask = matching_rows[col].astype(str).str.contains('eu_5', case=False, na=False)
        if mask.any():
            eu_5_rows = matching_rows[mask]
            print(f"Found 'eu_5' in column '{col}'")
            break
    
    if eu_5_rows is not None and len(eu_5_rows) > 0:
        print(f"\n{'='*80}")
        print("ROW WITH DATE 8/12/2025 AND eu_5:")
        print("="*80)
        print(eu_5_rows)
    else:
        print(f"\nNo 'eu_5' found in rows with date {target_date}")
        print("Showing all matching date rows:")
        print(matching_rows)


C:\Users\nhphuong\AppData\Local\Temp\ipykernel_21672\280054810.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], errors='coerce')


No rows found with date 8/12/2025

Trying to find any row with 'eu_5'...


In [7]:
# More direct approach: Search for rows matching both conditions
# Convert all date columns to datetime if possible
df_search = df.copy()

# Try to identify date and eu_5 columns more systematically
print("="*80)
print("SEARCHING FOR ROW WITH DATE 8/12/2025 AND eu_5")
print("="*80)

# Find all columns that might contain the date
date_mask = None
for col in df_search.columns:
    try:
        # Try converting to datetime
        temp_col = pd.to_datetime(df_search[col], errors='coerce')
        # Check if any value matches our target date
        date_match = (temp_col.dt.year == 2025) & (temp_col.dt.month == 8) & (temp_col.dt.day == 12)
        if date_match.any():
            date_mask = date_match
            print(f"Found date 8/12/2025 in column: '{col}'")
            break
    except:
        # Try string matching
        str_col = df_search[col].astype(str)
        if any('8/12/2025' in str_val or '2025-08-12' in str_val or '08/12/2025' in str_val 
               for str_val in str_col.head(100)):
            date_mask = str_col.str.contains('8/12/2025|2025-08-12|08/12/2025', case=False, na=False, regex=True)
            if date_mask.any():
                print(f"Found date 8/12/2025 in column: '{col}' (string match)")
                break

# Find all columns that might contain eu_5
eu_5_mask = None
eu_5_col = None
for col in df_search.columns:
    str_col = df_search[col].astype(str)
    if str_col.str.contains('eu_5', case=False, na=False).any():
        eu_5_mask = str_col.str.contains('eu_5', case=False, na=False)
        eu_5_col = col
        print(f"Found 'eu_5' in column: '{col}'")
        break

# Combine both conditions
if date_mask is not None and eu_5_mask is not None:
    combined_mask = date_mask & eu_5_mask
    if combined_mask.any():
        result_row = df_search[combined_mask]
        print(f"\n{'='*80}")
        print("FOUND ROW WITH DATE 8/12/2025 AND eu_5:")
        print("="*80)
        print(result_row.to_string())
    else:
        print("\nNo row found with both conditions simultaneously.")
        print("Showing rows with date match:")
        print(df_search[date_mask])
        print("\nShowing rows with eu_5 match:")
        print(df_search[eu_5_mask])
elif date_mask is not None:
    print(f"\nFound {date_mask.sum()} row(s) with date 8/12/2025, but no 'eu_5' column found.")
    print("Rows with date match:")
    print(df_search[date_mask])
elif eu_5_mask is not None:
    print(f"\nFound {eu_5_mask.sum()} row(s) with 'eu_5', but no date 8/12/2025 found.")
    print("Rows with eu_5 match:")
    print(df_search[eu_5_mask])
else:
    print("\nCould not find either condition. Showing first few rows for reference:")
    print(df_search.head())


SEARCHING FOR ROW WITH DATE 8/12/2025 AND eu_5
Found date 8/12/2025 in column: 'datetime'

Found 630 row(s) with date 8/12/2025, but no 'eu_5' column found.
Rows with date match:
                  datetime       date                time  year  month  day  \
0      2025-08-12 17:07:56 2025-08-12 2026-01-14 17:07:56  2025      8   12   
1      2025-08-12 17:17:57 2025-08-12 2026-01-14 17:17:57  2025      8   12   
2      2025-08-12 17:27:58 2025-08-12 2026-01-14 17:27:58  2025      8   12   
3      2025-08-12 17:37:59 2025-08-12 2026-01-14 17:37:59  2025      8   12   
4      2025-08-12 17:48:00 2025-08-12 2026-01-14 17:48:00  2025      8   12   
...                    ...        ...                 ...   ...    ...  ...   
251349 2025-08-12 23:12:50 2025-08-12 2026-01-14 23:12:50  2025      8   12   
251350 2025-08-12 23:22:51 2025-08-12 2026-01-14 23:22:51  2025      8   12   
251351 2025-08-12 23:32:52 2025-08-12 2026-01-14 23:32:52  2025      8   12   
251352 2025-08-12 23:42:53 2025

In [9]:
# Search for row with date 8/12/2025 and eu = 5
print("="*80)
print("SEARCHING FOR ROW WITH DATE 8/12/2025 AND eu = 5")
print("="*80)

# Filter for date 8/12/2025
df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce')
date_mask = (df['datetime'].dt.year == 2025) & (df['datetime'].dt.month == 8) & (df['datetime'].dt.day == 12)

# Filter for eu = 5
eu_mask = df['eu'] == 2

# Combine both conditions
combined_mask = date_mask & eu_mask

if combined_mask.any():
    result_row = df[combined_mask]
    print(f"\nFound {len(result_row)} row(s) with date 8/12/2025 and eu = 5")
    print("="*80)
    print(result_row.to_string())
else:
    print("\nNo row found with both conditions simultaneously.")
    print(f"\nRows with date 8/12/2025: {date_mask.sum()}")
    print(f"Rows with eu = 5: {eu_mask.sum()}")
    
    # Show sample of each
    if date_mask.any():
        print("\nSample rows with date 8/12/2025:")
        print(df[date_mask][['datetime', 'eu']].head())
    if eu_mask.any():
        print("\nSample rows with eu = 5:")
        print(df[eu_mask][['datetime', 'eu']].head())


SEARCHING FOR ROW WITH DATE 8/12/2025 AND eu = 5

Found 84 row(s) with date 8/12/2025 and eu = 5
                  datetime       date                time  year  month  day  hour  minute  second  area  eu  SHT0_temp  SHT0_humidity  SHT0_F1  SHT0_F2  SHT0_F3  SHT0_F4  SHT0_F5  SHT0_F6  SHT0_F7  SHT0_F8  SHT0_F9  SHT0_F10  SHT0_F11  SHT0_F12  SHT1_temp  SHT1_humidity  SHT1_F1  SHT1_F2  SHT1_F3  SHT1_F4  SHT1_F5  SHT1_F6  SHT1_F7  SHT1_F8  SHT1_F9  SHT1_F10  SHT1_F11  SHT1_F12          file  sht0_irradiance(μW/cm²)     sht0_lux  sht1_irradiance(μW/cm²)     sht1_lux
12252  2025-08-12 17:07:11 2025-08-12 2026-01-14 17:07:11  2025      8   12    17       7      11     1   2      38.41          38.37    635.0    880.0   1122.0   1398.0   8270.0   6684.0   1557.0   1548.0   1823.0    1922.0    8275.0    6685.0      33.27          49.69    438.0    571.0    726.0    900.0   5026.0   2262.0    974.0   1009.0   1102.0    1273.0    5028.0    2262.0  25Argv48.txt             27388.698849  25827.539